In [1]:
# signature = "{'dag_id': 'smc_daily_import', 'task_id': 'weblog_index_update', 'owner': 'icarus_so', 'nb_path': '/app/ws-ic/20200622_ssl_schema/update_domain_index.ipynb', 'execution_date': '2022-03-12'}"

#### 0. Initialization

In [25]:
from smtutil import hiveutil
import pandas as pd

In [2]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [3]:
sc, spark = hiveutil.get_sc_spark()

In [4]:
hive_db_name = 'weblog_v2'

In [7]:
# spark.sql(f"CREATE TABLE {hive_db_name}.t_domain_index LIKE {hive_db_name}.t_domain_index_new")

#### 1. Update Domain Index

##### 1.1. Calc domain - count(*) of the last date

In [5]:
spark.sql("MSCK REPAIR TABLE prd_biz_summ_vw.vw_idp_ssl_sed_call_summ")

DataFrame[]

In [6]:
spark.sql("REFRESH TABLE prd_biz_summ_vw.vw_idp_ssl_sed_call_summ")

DataFrame[]

In [7]:
# spark.sql(f"REFRESH TABLE {hive_db_name}.t_domain_index_new")

DataFrame[]

In [7]:
max_date_id = spark.sql("""
select max(date_id) as max_date_id
from prd_biz_summ_vw.vw_idp_ssl_sed_call_summ
""").first()['max_date_id']

In [8]:
spark.sql("""
select
    ssl_cert_domain,
    count(*) as c
from prd_biz_summ_vw.vw_idp_ssl_sed_call_summ
where date_id={max_date_id}
group by ssl_cert_domain
""".format(
    max_date_id=max_date_id
)).cache().createOrReplaceTempView("t_domain_count")

In [26]:
spark.sql("""
select
    ifnull(t_domain_count.ssl_cert_domain, t_domain_index.domain) as domain,
    t_domain_index.domain_index as o_domain_index,
    row_number() over (partition by 1 order by t_domain_index.domain_index nulls last) as domain_index
from t_domain_count
full outer join weblog.t_domain_index
    on t_domain_count.ssl_cert_domain = t_domain_index.domain
order by c desc
""").cache().createOrReplaceTempView("t_domain_index_new")

In [11]:
spark.table('t_domain_index_new').write.mode('overwrite').saveAsTable(f'{hive_db_name}.t_domain_index_new')

#### 2. Update t_domain_dictionary

In [12]:
spark.sql("""
with t_domain_dictionary_new as (
    select
        '<Unclassified>' as tag_gp,
        '<Unclassified>' as tag,
        t_domain_index_new.domain as domain,
        'domain' as agg_level,
        domain_index as agg_value
    from t_domain_index_new
    left join weblog.t_domain_dictionary
        on t_domain_index_new.domain_index = t_domain_dictionary.agg_value
    where agg_value is null
)
select * from t_domain_dictionary_new union all
select * from weblog.t_domain_dictionary
""").write.mode('overwrite').saveAsTable(f'{hive_db_name}.t_domain_dictionary_new')

#### 3. Rename New Tables

##### 3.1. t_domain_index_new

In [13]:
spark.sql(f"""drop table if exists {hive_db_name}.t_domain_index_backup""")
spark.sql(f"""ALTER table {hive_db_name}.t_domain_index RENAME TO {hive_db_name}.t_domain_index_backup""")
spark.sql(f"""ALTER table {hive_db_name}.t_domain_index_new RENAME TO {hive_db_name}.t_domain_index""")

AnalysisException: "Table or view 't_domain_index' not found in database 'weblog_v2';"

##### 3.2. t_domain_dictionary_new

In [ ]:
spark.sql(f"""drop table if exists {hive_db_name}.t_domain_dictionary_backup""")
spark.sql(f"""ALTER table {hive_db_name}.t_domain_dictionary RENAME TO {hive_db_name}.t_domain_dictionary_backup""")
spark.sql(f"""ALTER table {hive_db_name}.t_domain_dictionary_new RENAME TO {hive_db_name}.t_domain_dictionary""")

In [13]:
spark.stop()
sc.stop()